# Лабораторная работа №1

## Задания

Самостоятельно написать код, реализующий искусственный нейрон с сигма-функцией активации, и возможность строить на его основе многослойные сети. Код должен также реализовывать градиентный спуск и обратное распространение ошибки.

На основе вашего кода:

1. Решить задачу  классификации датаcета Iris одним нейроном.
2. Решить задачу  классификации датаcета Iris одним  нейросетью из 2 слоев по 10 нейронов в слое.
3. Отрисовать разделяющую линию для обеих моделей. Сравнить метрики классификации.

## Реализация

### Библиотеки

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

### Функция активации

В качестве неё берём сигма-функцию

In [2]:
def activation(x):
    return 1 / (1 + np.exp(-x))

In [3]:
def der_activation(x):  # производная функции активации (уже посчитан вывод)
    return x * (1 - x)

### Функция ошибок

Т.к. это задача классификации, то будет использоваться функция ошибок Categorical Cross-Entropy Loss

Её формула:

```
-sum_i(sum_j(y_ij * log(z_ij) + (1 - y_ij) * log(1 - z_ij)))
```

где

* 1 <= i <= N, N - кол-во входных данных
* 1 <= j <= C, C - кол-во классов
* y_ij - индикатор 0 или 1, что класс j является правильным для элемента i
* z_ij - предсказанная вероятность того, что у элемента i класс j
* log - логарифм
* sum_i - сумма по i
* sum_j - сумма по j

In [4]:
def loss(y_true, y_pred):
    N = len(y_true)
    C = len(y_true[0])

    total_sum = 0

    for i in range(N):
        value_sum = 0
        
        for j in range(C):
            p = y_pred[i][j]
            t = y_true[i][j]

            if p == 0:
                value_sum += t

            elif p == 1:
                value_sum += (1-t)

            else:
                value_sum += t * np.log(p) + (1 - t) * np.log(1 - p)
            
        total_sum += value_sum
            
    return -total_sum

In [5]:
def der_mult(y_true, y_pred):  # произведение производных функции ошибок и функции активации
    return y_pred - y_true

### Слой нейронной сети

Хранит матрицу весов нейронов, матрицу сдвигов нейронов, матрицу ошибок и вывод нейронов

In [6]:
class Layer:
    def __init__(self, neuron_count):
        self.neuron_count = neuron_count
        self.weights = None  # матрица весов нейронов N x W, где N - кол-во нейронов в слое, а W - кол-во входных данных в слой
        self.biases = None  # вектор сдвигов нейронов N x 1, где N - кол-во нейронов в слое
        self.output = None  # вектор вывода N x 1
        self.error = None  # вектор ошибок N x 1, нужен для обратного распространения ошибки в градиентном спуске

    
    def setup_layer(self, input_count):
        self.weights = np.zeros((self.neuron_count, input_count))

        for i in range(self.neuron_count):  # изначальные веса равны N(0, 2 / (n + w)), где n - кол-во нейронов, w - кол-во входов, N - нормальное распределение
            for j in range(input_count):
                self.weights[i][j] = np.random.normal(0, 2 / (self.neuron_count + input_count))  
                
        self.biases = np.zeros((self.neuron_count, 1))  # изначальный сдвиг равен 0

    
    def input_output(self, input_values):
        self.output = activation((self.weights @ input_values) + self.biases)  # вектор вывода, здесь F(WX + B), где X - вектор входных данных, W - матрица весов, B - вектор сдвигов
        
        return self.output

### Нейронная сеть

Сеть состоит из слоёв, здесь определяется её поведение

X и y нужно преобразовать перед тем, как работать с ними

Предположим, для задачи классификации есть 4 возможных вывода (0, 1, 2, 3) и y выглядит так:

```
| y |
|---|
| 0 |
| 2 |
| 1 |
| 2 | 
| 3 |
| 1 |
```

Нейроны выдают значения от 0 до 1, и чтобы классификация прошла правильно, преобразуем данные в матрицу вхождений

```
| y |     |    y    |
|---|     |---------|
| 0 | --> | 1 0 0 0 |
| 2 | --> | 0 0 1 0 |
| 1 | --> | 0 1 0 0 |
| 2 | --> | 0 0 1 0 |
| 3 | --> | 0 0 0 1 |
| 1 | --> | 0 1 0 0 | 
```

Также нужно нормализировать входные данные в пределах (-1, 1), тогда mean = 0, а var = 1

In [7]:
class ClassificationNetwork:
    def __init__(self, layers=[]):
        self.layers = layers

    
    def back_propogation(self, true_values):  # обратное распространение ошибки
        for layer in range(len(self.layers) - 1, -1, -1):  # идём в обратном порядке, в этом суть метода
            cur_layer = self.layers[layer]
            
            if layer == len(self.layers) - 1:
                # Для выходного слоя вектор ошибок будет выглядеть так: L'(t, o) * f'(i), где:
                # L' - производная функции ошибок, f' - производная функции активации
                # o - вывод значений нынешнего слоя, t - истинный вывод, i - вывод значений предыдущего слоя
                # Важное замечание: у функции активации сигма, производная равна f(i)(1 - f(i))
                # Т.к. мы уже знаем, что o = f(i), то тогда f'(i) = o(1 - o)
                
                cur_layer.error = der_mult(true_values, cur_layer.output)

            else:
                # Для остальных слоёв вектор ошибок будет такой: TD * f'(i), где:
                # D - вектор ошибок следующего слоя, T - транспонированная матрица весов следующего слоя
                # f' - производная функции активации, o - вывод значений нынешнего слоя, i - вывод значений предыдущего слоя
                # Важное замечание: у функции активации сигма, производная равна f(i)(1 - f(i))
                # Т.к. мы уже знаем, что o = f(i), то тогда f'(i) = o(1 - o)
                
                next_layer = self.layers[layer + 1]
                cur_layer.error = (next_layer.weights.transpose() @ next_layer.error) * der_activation(cur_layer.output)
                
    
    def gradient_descent(self, input_values, learning_rate):  # градиентный спуск
        for layer in range(len(self.layers)):  # теперь идём по порядку, так называемый forward progogation     
            cur_layer = self.layers[layer]

            # Изменение сдвигов равно E*l, где l - скорость обучения, E - вектор ошибок

            delta_biases = cur_layer.error * learning_rate 

            # Изменение весов равно ET*l, где l - скорость обучения, E - вектор ошибок, T - транспонированная матрица вводных данных слоя

            delta_weights = None

            if layer == 0:
                delta_weights = (cur_layer.error @ input_values.transpose()) * learning_rate

            else:
                prev_layer = self.layers[layer - 1]
                delta_weights = (cur_layer.error @ prev_layer.output.transpose()) * learning_rate

            cur_layer.weights -= delta_weights
            cur_layer.biases -= delta_biases


    def normalise(self, X):  # нормализация входных данных
        normalised = np.zeros(X.shape)
    
        for i in range(normalised.shape[1]):        
            normalised[:, i] = (X[:, i] - X[:, i].mean()) / (X[:, i].max() - X[:, i].min())
    
        return normalised


    def one_hot_transform(self, y, classes_count):  # преобразование в матрицу вхождений
        transformed = np.zeros((len(y), classes_count))
    
        for i in range(len(y)):  # исходит из предположения, что классы будут пронумерованы по порядку, начиная от 0
            transformed[i][y[i]] = 1
    
        return transformed

        
    def fit(self, X, y, learning_rate=0.1, epochs=1000, verbose=False, early_stopping=None):        
        X_train = self.normalise(X)
        y_train = self.one_hot_transform(y, self.layers[-1].neuron_count)  # столько классифицируем, сколько нейронов на последнем слое
        
        for layer in range(len(self.layers)):  # инициализация слоёв
            if layer == 0:
                self.layers[layer].setup_layer(len(X_train[0]))  # первый слой - входной, у него кол-во входных данных равно кол-ву вводимых атрибутов

            else:
                self.layers[layer].setup_layer(self.layers[layer-1].neuron_count)  # у каждого последующего слоя кол-во входных данных равно кол-ву нейронов на предыдущем слое

        last_error = None

        for epoch in range(epochs):  # начало обучения
            y_pred = []
            
            for value in range(len(X_train)):
                input_values = np.array([X_train[value]]).transpose()  # входные данные в виде вектор-столбца
                true_values = np.array([y_train[value]]).transpose()  # истинные данные в виде вектор-столбца
                
                neuron_output = input_values  # в первый слой идут входные данные

                for layer in range(len(self.layers)):       
                    neuron_output = self.layers[layer].input_output(neuron_output)

                y_pred.append(neuron_output.transpose()[0])  # значение последнего слоя и будет предсказанным значением
                
                self.back_propogation(true_values)  # считаем обратное распространение ошибки
                self.gradient_descent(input_values, learning_rate)  # выполняем градиентный спуск

            y_pred = np.array(y_pred)
            error = loss(y_train, y_pred)

            if verbose:
                print(f"Epoch {epoch+1} out of {epochs}, current error is {error}")

            if early_stopping is not None:  # early stopping - минимальное значение разницы ошибок последних двух итераций, меньше которого обучение останавливается
                if last_error is not None:
                    if np.abs(error - last_error) < early_stopping:
                        if verbose:
                            print("Stopping early")
                        return

                last_error = error

    
    def predict(self, X):
        X_test = self.normalise(X)
        y_pred = []  # предсказанные значения

        for value in range(len(X_test)):            
            neuron_output = np.array([X_test[value]]).transpose()  # входные данные в виде вектор-столбца

            for layer in range(len(self.layers)):    
                neuron_output = self.layers[layer].input_output(neuron_output)

            probab = neuron_output.transpose()[0] # значение последнего слоя и будет предсказанным значением - вероятности для каждого класса
            y_pred.append(np.argmax(probab))  # выбираем класс с наибольшей вероятностью

        return np.array(y_pred)  # получили предсказание

### Датасет

In [8]:
df = load_iris()
X, y = df["data"], df["target"]
classes_count = len(df["target_names"])  # кол-во классов

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
y_test

array([2, 1, 1, 1, 2, 1, 0, 1, 1, 0, 2, 0, 2, 1, 0, 1, 1, 0, 2, 1, 2, 2,
       2, 2, 1, 1, 0, 0, 1, 2])

### Сеть из 1 слоя с 1 нейроном

In [10]:
model1 = ClassificationNetwork([
    Layer(1),
    Layer(classes_count)  # кол-во нейронов на выходном слою равно кол-ву типов классификации
])

In [11]:
model1.fit(X_train, y_train, verbose=False, learning_rate=0.1, epochs=5000, early_stopping=10**-3)

In [12]:
y_pred1 = model1.predict(X_test)
y_pred1

array([1, 1, 1, 1, 1, 2, 0, 1, 1, 0, 2, 0, 2, 1, 0, 1, 1, 0, 2, 1, 2, 2,
       2, 2, 1, 1, 0, 0, 2, 2])

### Сеть из 2 слоёв по 10 нейронов

In [13]:
model2 = ClassificationNetwork([
    Layer(10),
    Layer(10),
    Layer(classes_count)  # кол-во нейронов на выходном слою равно кол-ву типов классификации
])

In [14]:
model2.fit(X_train, y_train, verbose=False, learning_rate=0.1, epochs=5000, early_stopping=10**-3)

In [15]:
y_pred2 = model2.predict(X_test)
y_pred2

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 0, 2, 1, 0, 1, 1, 0, 2, 1, 2, 2,
       2, 2, 1, 0, 0, 0, 2, 2])